# Detecting Financial Fraud using AutoML Tables

**Import the needed python libraries**

In [ ]:
import pandas as pd
from google.cloud import automl_v1beta1 as automl
from google.cloud import bigquery
import pandas as pd
import google.cloud.automl_v1beta1.proto.data_types_pb2 as data_types

**Enter Variables used for the Script**

In [ ]:
# enter GCP ProjectID
PROJECT_ID =   # e.g. "fraud-demo-sled"

# enter compute region 
COMPUTE_REGION = "us-central1" #only us-central1 is supported for now

BQ_DATASET_NAME =  # e.g. "fin_fraud_data"

BQ_FRAUD_TABLE_NAME =  # e.g. "PS_2017"

main_table_name = '.'.join([PROJECT_ID, BQ_DATASET_NAME, BQ_FRAUD_TABLE_NAME])

training_table_name = '.'.join([PROJECT_ID, BQ_DATASET_NAME, "training"])

prediction_table_name = '.'.join([PROJECT_ID, BQ_DATASET_NAME, "actual"])

## Set up Bigquery Table to be imported into AutoML Tables

**Connect to Bigquery Client**

In [ ]:
bq_client = bigquery.Client(project = PROJECT_ID)

**Notes: Fraud cases are very rare. In this dataset there are 6,362,620 transactions. Only 8,213 are fraud.** <br>
This means that **99.87%** of transactions are not fraud. If we were to build a model and predict "not fraud" every time, that model will have a **"99.87%" accuracy!!!**. That is very accurate, but it would not be useful for our case. <br> 
If we were to leave a model with it's default settings, the model would predict that everything is "not fraud" by default. <br>
In order to account for the rare fraud occurances, we can either:
- **Random Oversampling:** create duplicates of of the fraud cases (to get close to the same number of not fraud cases) 
- **Random Undersampling:** remove "not fraud" cases so that the fraud and not fraud cases are close to even
- **Give rare case higher weight:** add a a weight to the fraud cases so that the model views them as more important <br>

I thought that the number disparity between fraud and not fraud were too great to over or undersample. If we oversampled the ML model may see the same cases enough that it would identify those exact features as being fraudlent (i.e. a transaction of $1,334 is always fraud because the model has see it 1,000 times). While I thought that undersampling would lose too much data (we would go from 6,000,000 cases to less than 20,000. I ended up weighing cases identified as fraud at a much higher rate to tell the ML model that there will be a *big penealty for missing a fraudulent transaction* as compared to getting a "not fraud" wrong. <br>

The proportion of fraud to not fraud is 773.7. To keep it simple, I weigh all fraud cases as 800 in this demo.

**Note:** In the dataset, the max step (number of hours) is 743. Since the steps move toward the future (0 hours to 743 hours), I decided that it was best to split the data in a way that train to predict future events.

In [ ]:
max_step = 743

**Split and Weight columns are created below** <br>

**The dataset is split in four ways:**\
**1. Train:** The first 70% of hours data is our training set\
**2. Validation:** the next 10% of hours is our validation set\
**3. Test:** the next 10% of hours is our test set\
**4. Prediction:** The last 10% of hours is not used by the model, but instead used by us for prediction <br>


In [ ]:
table_id = training_table_name
job_config = bigquery.QueryJobConfig(destination=table_id)

sql = """SELECT
  CASE
    WHEN step < CEIL({MAX_STEP}*.70) THEN "TRAIN"
    WHEN step < CEIL({MAX_STEP}*.80) AND step >= CEIL({MAX_STEP}*.70) THEN "VALIDATE"
    WHEN step < CEIL({MAX_STEP}*.90) AND step >= CEIL({MAX_STEP}*.80) THEN "TEST" END as split,
  CASE
    WHEN isfraud = 1 THEN 800
    WHEN isfraud = 0 THEN 1
    END as weight
    , 
    *
FROM
  `{MAIN_TABLE}`
WHERE step < CEIL({MAX_STEP}*.90)
""".format(MAX_STEP=max_step,MAIN_TABLE = main_table_name)


# Start the query, passing in the extra configuration.
query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_id)

In [ ]:
table_id = prediction_table_name
job_config = bigquery.QueryJobConfig(destination=table_id)

sql = """
SELECT *
FROM
  `{MAIN_TABLE}`
WHERE step >= CEIL({MAX_STEP}*.90)
""".format(MAX_STEP=max_step)

# Start the query, passing in the extra configuration.
query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_id)

## Train Model and Make Predictions with 

**Connect to AutoML Tables Client**

In [ ]:
tables_client = automl.TablesClient(project=PROJECT_ID)

In [ ]:
# enter AutoML dataset name
DATASET_DISPLAY_NAME = "fin_fraud_data"

# enter AutoML Model Display name
MODEL_DISPLAY_NAME = "fraud_detection_model"

**Create AutoML Dataset**

In [ ]:
try:
    dset = tables_client.get_dataset(dataset_display_name= DATASET_DISPLAY_NAME)
    print("dataset exist")
except: 
    dataset = tables_client.create_dataset(DATASET_DISPLAY_NAME)

**Import the dataset from Bigquery** <br> Note: Datasets can also be imported from GCS (as a csv) <br>
**The models take between 10-30 minutes to import**

In [ ]:
response = tables_client.import_data(
    dataset_display_name= DATASET_DISPLAY_NAME 
    bigquery_input_uri="bq://" + training_table_name
)

**Tell AutoML Tables how to split the data into traning, validation, and tetsing by pointing it to the "split" column that we created earlier** 

In [ ]:
tables_client.set_test_train_column(dataset_display_name=DATASET_DISPLAY_NAME,column_spec_display_name="split")

**View Dataset Specs, Update any types that were imported incorrectly** <br>
Exmaple: Chnage if a FLOAT was imported as a STRING

In [ ]:
dataset_spec = tables_client.get_dataset(dataset_display_name=DATASET_DISPLAY_NAME)

list_table_specs_response= tables_client.list_table_specs(dataset_display_name = DATASET_DISPLAY_NAME)
table_specs = [s for s in list_table_specs_response]

# List column specs.
list_column_specs_response = tables_client.list_column_specs(dataset_display_name= DATASET_DISPLAY_NAME)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print Features and data_type.
features = [(key, data_types.TypeCode.Name(value.data_type.type_code)) 
            for key, value in column_specs.items()]
print('Feature list:\n')
for feature in features:
    print(feature[0],':', feature[1])

**Update the weight for transactions that were identified as Fraud!!!** <br>


In [ ]:
weight_column_name = weight 
update_weight_column = tables_client.update_column_spec(
    dataset_display_name= DATASET_DISPLAY_NAME
    column_spec_display_name= weight_column_name,
    type_code="FLOAT64",
    nullable=False)

set_weight_column = tables_client.set_weight_column(dataset_display_name=DATASET_DISPLAY_NAME, column_spec_display_name= weight_column_name)

**Set the target column**<br>
The target column is IsFraud.

In [ ]:
set_target = tables_client.set_target_column(dataset_display_name=DATASET_DISPLAY_NAME, column_spec_display_name= "isFraud")

**Train the model** <br>
The minimum training time is 1 hour. The max is 10 hours. <br>
Trianing should automatically stop at the time when more training does not improve accuracy <br>
We use 2 hours for this demo (it cost around \\$40 to build) <br>
**The model will take 2 hours to build! You must wait 2 hours for predictions**

In [ ]:
model_train_hours = 2
create_model = tables_client.create_model(model_display_name = MODEL_DISPLAY_NAME,
                           dataset_display_name=DATASET_DISPLAY_NAME,
                           train_budget_milli_node_hours= model_train_hours * 1000)

Your model can be seen in the model list when it is ready. 

In [ ]:
model_list = tables_client.list_models()
for i in model_list:
    print(i.display_name)

**Deploy Model**<br>
Deployed models are only used for online prediction, but it take around 10 minutes to delpoy a model. Deploy now so that you can do the online predictions faster later.   

In [ ]:
model_dep_rep = tables_client.deploy_model(model_display_name=MODEL_DISPLAY_NAME)

## Make some predictions

### Batch Predictions
**Predict a large number of transactions in batch** <br>
- A model does not have to be deployed for "batch" predictions
- Batch predictions cost less than online predictions
- Batch predictions take around 2-5 minutes to return results, while online is real-time

In [ ]:
batch_predict = tables_client.batch_predict(model_display_name= MODEL_DISPLAY_NAME,
                            bigquery_input_uri= 'bq://' + prediction_table_name,
                            bigquery_output_uri= 'bq://' + PROJECT_ID)

### Online (Real-time) Predictions

**Fill in inputs and make a prediction. This cell should return you will be able to view the raw output.**

In [ ]:
input = {"step": 675, "type": "Debit",
          "amount": 10000, "nameOrig": "C12434555",
          "oldbalanceOrg": 10000, "newbalanceOrig": 0,
          "nameDest": "M12345544", "oldbalanceDest": 10000,
          "newbalanceDest": 0, "isFlaggedFraud": "1"}

prediction = tables_client.predict(inputs=inputs, model_display_name= MODEL_DISPLAY_NAME)

prediction

**The end user can choose what probablity theshold tha they want to act on. By default, many models say that some is true if the probability is more than 50%. In reality, the probabilites close to 50% are more ambiguous, while the ones closer to 0% or 100% are more certain.**

In [ ]:
prediction_percent = round(prediction.payload[1].tables.score * 100, 2)
print("The model predicts that there is a {}% chance that the transaction is fraudulent.".format(prediction_percent) )